<a href="https://www.kaggle.com/code/aleksandrmorozov123/computer-vision-excercises?scriptVersionId=118887012" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***My computer vision exercises. I begin study this theme***

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

def read_and_decode (filename, reshape_dims):
    # read the file
    img = tf.io.read_file (filename = '../input/flowers-recognition/flowers/rose/10090824183_d02c613f10_m.jpg')
    # convert the compressed string to a 3D unit8 tensor
    img = tf.image.decode_jpeg (img, channels = 3)
    # convert 3D unit8 to floats in the [0,1] range
    img = tf.image.convert_image_dtype (img, tf.float32)
    # resize the image to the desired size
    return tf.image.resize (img, reshape_dims)

# visualize image data
IMG_HEIGHT = 256
IMG_WIDTH = 128
def show_image (filename):
    img = read_and_decode (filename, [IMG_HEIGHT, IMG_WIDTH])
    plt.imshow (img.numpy ())
    
show_image ('../input/flowers-recognition/flowers/rose/10090824183_d02c613f10_m.jpg')

***Accessing and manipulating pixels***

In [ ]:
import numpy as np
import cv2

# read the rose image and load it into a variable rose_image
rose_image = cv2.imread ('../input/flowers-recognition/flowers/sunflower/1008566138_6927679c8a.jpg')

# access a specific pixel using the coordinate based access from the matrix
pixel = rose_image [150, 170]

# see what color space that pixel represents
print (pixel)

# lets change the pixel color value to blue
rose_image [150, 170] = (255, 0, 0)

# lets change the pixel color value to blue in a region range as against
rose_image [120:150, 150:170] = (0, 255, 0)

cv2.line (rose_image, (25, 21), (100, 100), (255, 0, 0), 5)

cv2.rectangle (rose_image, (25, 51), (100, 100), (0, 255, 0), 2)
cv2.circle (rose_image, (50, 50), 50, (0, 0, 255), -1)

**Compute convolutions - 1 dimensional**

In [ ]:
import numpy as np
def conv1d (x, w, p = 0, s = 1):
    w_rot = np.array (w[:: -1])
    x_padded = np.array (x)
    if p > 0:
        zero_pad = np.zeros (shape = p)
        x_padded = np.concatenate ([zero_pad, x_padded, zero_pad])
    res = []
    for i in range (0, int (len (x)/s), s):
        res.append (np.sum (x_padded [i:i + w_rot.shape [0]] * w_rot))
    return np.array (res)

# run the test
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print ('Conv1d Implementation:', conv1d (x, w, p = 2, s = 1))

print ('Numpy Results:', np.convolve (x, w, mode = 'same'))

**Compute convultions - 2 dimensional**

In [ ]:
import scipy.signal

def conv2d (X, W, p = (0, 0), s = (1, 1)):
    W_rot = np.array (W) [::-1, ::-1]
    X_orig = np.array (X)
    n1 = X_orig.shape [0] + 2*p [0]
    n2 = X_orig.shape [1] + 2*p [1]
    X_padded = np.zeros (shape = (n1, n2))
    X_padded [p[0]:p[0] + X_orig.shape [0], p[1]:p[1] + X_orig.shape [1]] = X_orig
    res = []
    for i in range (0, int ((X_padded.shape [0] - W_rot.shape [0])/s[0]) +1, s [0]):
        res.append ([])
        for j in range (0, int ((X_padded.shape [1] - W_rot.shape [1]) /s[1]) +1, s[1]):
            X_sub = X_padded [i:i+W_rot.shape [0],j:j + W_rot.shape [1]]
            res [-1].append (np.sum (X_sub * W_rot))
    return (np.array (res))

X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

print ('Conv2d Implementation:\n', conv2d (X, W, p = (1, 1), s = (1, 1)))

print ('Scipy ResultsL \n', scipy.signal.convolve2d (X, W, mode = 'same'))

**Deep learning with fastai**

In [ ]:
from fastai.vision.all import *

results = ('../input/bears-fastai-2021/bears')
len (results)

In [ ]:
dest = ('../input/bears-fastai-2021/bears/black/00000000.jpg')
im = Image.open (dest)
im.to_thumb (128, 128)

In [ ]:
bear_types = 'black', 'grizzly', 'teddy'
path = Path ('../input/bears-fastai-2021/bears')
if not path.exists ():
    path.mkdir ()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir (exist_ok = True)
        results = ('../input/bears-fastai-2021/bears')

In [ ]:
fns = get_image_files (path)
fns

In [ ]:
class DataLoaders (GetAttr):
    def __init__ (self, *loaders): self.loaders = loaders
    def __getitem__ (self, i): return self.loaders [i]
    train, valid = add_props (lambda i, self: self [i])

In [ ]:
bears = DataBlock (blocks = (ImageBlock, CategoryBlock),
                  get_items = get_image_files,
                  splitter = RandomSplitter (valid_pct = 0.2, seed = 42),
                  get_y = parent_label, item_tfms = Resize (128))
dls = bears.dataloaders (path)
dls.valid.show_batch (max_n = 4, nrows = 1)

In [ ]:
bears = bears.new (item_tfms = Resize (128, ResizeMethod.Squish))
dls = bears.dataloaders (path)
dls.valid.show_batch (max_n = 4, nrows = 1)

In [ ]:
bears = bears.new (item_tfms = Resize (128, ResizeMethod.Pad, pad_mode = 'zeros'))
dls = bears.dataloaders (path)
dls.valid.show_batch (max_n = 4, nrows = 1)

In [ ]:
bears = bears.new (item_tfms = RandomResizedCrop (128, min_scale = 0.3))
dls = bears.dataloaders (path)
dls.train.show_batch (max_n = 4, nrows = 1, unique = True)

In [ ]:
bears = bears.new (item_tfms = Resize (128), batch_tfms = aug_transforms (mult = 2))
dls = bears.dataloaders (path)
dls.train.show_batch (max_n = 8, nrows = 2, unique = True)

In [ ]:
bears = bears.new (item_tfms = RandomResizedCrop (224, min_scale = 0.5),
                  batch_tfms = aug_transforms ())
dls = bears.dataloaders (path)
learn = vision_learner (dls, resnet18, metrics = error_rate)
learn.fine_tune (4)

In [ ]:
interp = ClassificationInterpretation.from_learner (learn)
interp.plot_confusion_matrix ()

In [ ]:
interp.plot_top_losses (5, nrows = 1)

In [ ]:
learn.export ()
path = Path ()
path.ls (file_exts ='.pkl')
learn_inf = load_learner (path/'export.pkl')
learn_inf.predict ('../input/bears-fastai-2021/bears/grizzly/00000004.jpg')

In [ ]:
learn_inf.dls.vocab

In [ ]:
from fastai.vision.all import *

# download a sample of MNIST that contains images digits
path = untar_data (URLs.MNIST_SAMPLE)
path.ls ()

In [ ]:
# print the items in directory 'train'
(path/'train').ls ()

In [ ]:
# look on folders '3' and '7' (this is targets)
threes = (path/'train'/'3').ls ().sorted ()
sevens = (path/'train'/'7').ls ().sorted ()
threes

In [ ]:
# print the image hand-written number from MNIST dataset 
im3_path = threes [1]
im3 = Image.open (im3_path)
im3

In [ ]:
# convert image in a NumPy array
array (im3)[4:10, 4:10]

In [ ]:
# convert a image in a PyTorch tensor
tensor (im3) [4:10, 4:10]

In [ ]:
# using a gradient to show how image is created
im3_t = tensor (im3)
df = pd.DataFrame (im3_t [4:15, 4:22])
df.style.set_properties (**{'font_size': '6pt'}).background_gradient ('Greys')

In [ ]:
# check the number of returned items
seven_tensors = [tensor (Image.open (o)) for o in sevens]
three_tensors = [tensor (Image.open (o)) for o in threes]
len (three_tensors), len (seven_tensors)

In [ ]:
# apply the show_image function
show_image (three_tensors [1])

In [ ]:
# images are floats and we divide by 255
stacked_sevens = torch.stack (seven_tensors).float ()/255
stacked_threes = torch.stack (three_tensors).float ()/255
stacked_threes.shape

In [ ]:
# length tensor is its rank
len (stacked_threes.shape)

In [ ]:
# tensor rank
stacked_threes.ndim

In [ ]:
# compute the average of that pixel over all images
mean3 = stacked_threes.mean (0)
show_image (mean3)

In [ ]:
# do the same thing for 7
mean7 = stacked_sevens.mean (0)
show_image (mean7)

In [ ]:
# pick 3 and measure its distance from ideal digit
a_3 = stacked_threes [1]
show_image (a_3)

In [ ]:
# measure the distance from 3 with mathematics
dist_3_abs = (a_3 - mean3).abs ().mean ()
dist_3_sqr = ((a_3 - mean3)**2).mean ().sqrt ()
dist_3_abs, dist_3_sqr

In [ ]:
# measure the distance from 7 with mathematics
dist_7_abs = (a_3 - mean7).abs ().mean ()
dist_7_sqr = ((a_3 - mean7)**2).mean ().sqrt ()
dist_7_abs, dist_7_sqr

In [ ]:
# calculate the mean squared error 
F.l1_loss (a_3.float (), mean7), F.mse_loss (a_3, mean7).sqrt ()

In [ ]:
# create numpy array
data = [[2,3,4], [5,6,7]]
arr = array (data)
tns = tensor (data)
arr

In [ ]:
# create tensor array
tns

In [ ]:
# select a row (index start with 0)
tns [0]

In [ ]:
# select a column
tns [:, 2]

In [ ]:
# use the maultiplication operation to array
tns * 2

In [ ]:
# tensor type
tns.type ()

In [ ]:
# change integer to float
tns * 2.7

In [ ]:
# create tensors
valid_3_tens = torch.stack ([tensor (Image.open (o))
                            for o in (path/'valid'/'3').ls ()])
valid_3_tens = valid_3_tens.float ()/255
valid_7_tens = torch.stack ([tensor (Image.open (o))
                            for o in (path/'valid'/'7').ls ()])
valid_7_tens = valid_7_tens.float ()/255
valid_3_tens.shape, valid_7_tens.shape

In [ ]:
# create a function that calculates the mean absolute error
def mnist_distance (a, b):
    return (a-b).abs ().mean ((-1, -2))
mnist_distance (a_3, mean3)

In [ ]:
# take same distance fucntion for comparing two single images
valid_3_dist = mnist_distance (valid_3_tens, mean3)
valid_3_dist, valid_3_dist.shape

In [ ]:
# create function for automatically do broadcastinf and be applied elementwise
def is_3 (x):
    return mnist_distance (x, mean3) < mnist_distance (x, mean7)
is_3 (a_3), is_3 (a_3).float ()

In [ ]:
# test it on the full validation set of 3s
is_3 (valid_3_tens)

In [ ]:
# calculate the accuracy for each of the 3s and 7s
accuracy_3s = is_3 (valid_3_tens).float ().mean ()
accuracy_7s = (1 - is_3 (valid_7_tens).float ()).mean ()
accuracy_3s, accuracy_7s, (accuracy_3s + accuracy_7s)/2

In [ ]:
# calculate vector argument for our function
xt = tensor ([2., 3., 9.]).requires_grad_ ()
xt

In [ ]:
# add sum to our function and return a scalar
def f (x):
    return (x**2).sum ()

yt = f (xt)
yt

In [ ]:
import matplotlib.pyplot as plt

# create a model of how the speed changes over time
time = torch.arange (0, 20).float (); time
speed = torch.randn (20)*3 + 0.75*(time - 9.5)**2 + 1
plt.scatter (time, speed)

In [ ]:
# collect the parameters in one argument of function and separate the input
def f (t, params):
    a, b, c = params
    return a * (t ** 2) + (b * t) + c

# use function mean squared error
def mse (preds, targets):
    return ((preds - targets)**2).mean ()

# initialize the parameters
params = torch.randn (3).requires_grad_ ()

# calculate the predictions
preds = f (time, params)

# create a fucntion to show how close our predictions are to our targets
def show_preds (preds, ax = None):
    if ax is None: 
        ax = plt.subplots ()[1]
        ax.scatter (time, speed)
        ax.scatter (time, to_np (preds), color = 'yellow')
        ax.set_ylim (-300, 100)
show_preds (preds)

In [ ]:
# calculate the loss
loss = mse (preds, speed)
loss

In [ ]:
# calculate the gradients
loss.backward ()
params.grad 

In [ ]:
# convert numbers to a more convenient format
params.grad * 1e-5

In [ ]:
params

In [ ]:
# step the weights
lr = 1e-5

def apply_step (params, prn = True):
    preds = f(time, params)
    loss = mse (preds, speed)
    loss.backward ()
    params.data -= lr * params.grad.data
    params.grad = None
    if prn:
        print (loss.item ())
        return preds
show_preds (preds)

In [ ]:
for i in range (10):
    apply_step (params)

In [ ]:
# MNIST loss function
train_x = torch.cat ([stacked_threes, stacked_sevens]).view (-1, 28*28)
train_y = tensor ([1] * len (threes) + [0] * len (sevens)).unsqueeze (1)
train_x.shape, train_y.shape

In [ ]:
# apply zip function
dset = list (zip (train_x, train_y))
x, y = dset [0]
x.shape, y

In [ ]:
# MNIST loss fucntion with zip
valid_x = torch.cat ([valid_3_tens, valid_7_tens]).view (-1, 28*28)
valid_y = tensor ([1] * len (valid_3_tens) + [0] * len (valid_7_tens)).unsqueeze (1)
valid_dset = list (zip (valid_x, valid_y))

def init_params (size, std = 1.0):
    return (torch.randn (size)* std).requires_grad_ ()
weights = init_params ((28*28, 1))
bias = init_params (1)
(train_x [0]*weights.T).sum () + bias

In [ ]:
# matrix ,ultiplication
def linear1 (xb):
    return xb@weights + bias
preds = linear1 (train_x)
preds

In [ ]:
# check our accuracy
corrects = (preds>0.0).float () == train_y
corrects

In [ ]:
corrects.float ().mean ().item ()

In [ ]:
# create a loss function to measure the distance between predictions and targets
trgts = tensor ([2, 0, 2])
prds = tensor ([0.9, 0.4, 0.2])
def mnist_loss (predictions, targets):
    return torch.where (targets == 1, 1 - predictions, predictions).mean ()
torch.where (trgts == 1, 1 - prds, prds)

In [ ]:
# load DataLoader for training model
coll = range (15)
dl = DataLoader (coll, batch_size = 5, shuffle = True)
list (dl)

In [ ]:
# create a simple dataset
ds = L (enumerate (string.ascii_lowercase))
ds

In [ ]:
dl = DataLoader (ds, batch_size = 6, shuffle = True)
list (dl)

***Image classification***

In [ ]:
# adding a dataset
from fastai.vision.all import *
path = Path ('../input/bears-fastai-2021/bears')

In [ ]:
# test out code and pick out one of filenames
fname = (path).ls ()[0]
re.findall (r'(.+)_\d+.jpg$', fname.name)

# labelling whole dataset
bears = DataBlock (blocks = (ImageBlock, CategoryBlock),
                  get_items = get_image_files,
                  splitter = RandomSplitter (seed = 42),
                  item_tfms = Resize (460),
                  batch_tfms = aug_transforms (size = 224, min_scale = 0.75))
dls = bears.dataloaders (path)
item_tfms = Resize (460)
batch_tfms = aug_transforms (size = 224, min_scale = 0.75)

# check our data
dls.show_batch (nrows = 1, ncols = 3)

In [ ]:
# summary dataset
bears.summary (path)

In [ ]:
# activate our model
x, y = dls.one_batch ()

# look at this contained dependent variable
y

In [ ]:
# use some random numbers with a standard deviation of 2 
acts = torch.randn ((6,2))*2
acts

In [ ]:
# pass through the sigmoid function
acts.sigmoid ()
(acts [:, 0] - acts [:, 1]).sigmoid ()

def softmax (x):
    return exp (x) / exp (x).sum (dim = 1, keepdim = True)

# check softmax function
sm_acts = torch.softmax (acts, dim = 1)
sm_acts

In [ ]:
# calculate loss fucntion
def mnist_loss (inputs, targets):
    inputs = inputs.sigmoid ()
    return torch.where (targets == 1, 1 - inputs, inputs).mean ()

# let's install the labels
targ = tensor ([0, 1, 0, 1, 1, 0])
sm_acts

In [ ]:
# use tensor indexing
idx = range (6)
sm_acts [idx, targ]

In [ ]:
# apply NLL (negative log likelihood)
-sm_acts [idx, targ]
F.nll_loss (sm_acts, targ, reduction = 'none')

In [ ]:
# apply cross-entropy loss
loss_func = nn.CrossEntropyLoss ()
loss_func (acts, targ), F.cross_entropy (acts, targ)

In [ ]:
# use a confusion matrix for appreciation our model
interp = ClassificationInterpretation.from_learner (learn)
interp.plot_confusion_matrix (figsize = (12, 12), dpi = 60)

In [ ]:
# improving our model
# apply the learning rate finder
from fastai.vision.learner import vision_learner
learn = vision_learner (dls, resnet34, metrics = error_rate)
learn,fine_tune (1, base_lr = 0.1)

In [ ]:
# build the learning rate finder 
learn = vision_learner (dls, resnet34, metrics = error_rate)
lr_min, lr_steep = learn.lr_find ()

In [ ]:
print (f"Minimum/10: {lr_min.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
learn = vision_learner (dls, resnet34, metrics = error_rate)
learn.fine_tune (2, base_lr = 3e-3)

In [ ]:
#unfreezing and transfer learning
learn = vision_learner (dls, resnet34, metrics = error_rate)
learn.fit_one_cycle (3, 3e-3)

In [ ]:
learn.unfreeze ()
learn.lr_find ()

In [ ]:
# train a suitable learning rate
learn.fit_one_cycle (6, lr_max = 1e-5)

In [ ]:
# set only the lowest layer of our net to a learning rate
learn = vision_learner (dls, resnet34, metrics = error_rate)
learn.fit_one_cycle (3, 3e-3)
learn.uinfreeze ()
learn.fit_one_cycle (12, lr_max = slice (1e-6, 1e-4))

In [ ]:
# visualize the training and validation loss
learn.recorder.plot_loss ()

In [ ]:
# try a ResNet-50
from fastai2.callback.fp16 import *
learn = vision_learner (dls, resnet50, metrics = error_rate).to_fp16 ()
learn.fine_tune (6, freeze_epochs = 3)